# 감정분석을 위한 데이터 훈련

In [2]:
# 감정분석을 위한 데이터 훈련
import csv

file = open("./ratings_train.txt",'r',encoding='cp949')
##긍정부정 직접 나눈 파일
file2 = open("./pos_neg_final.csv","r")

rdr= csv.reader(file2)

sentence=[]
label=[]

for index, line in enumerate(rdr):#직접 분류한걸 먼저 넣음(train데이터로 넣기 위함)
    sentence.append(line[1].strip())#sentence에 text를 넣음
    label.append(line[2].strip())#label에 긍정인지 부정인지 값을 넣음(1또는 0)

for index, line in enumerate(file):
    if index==0:#첫번째 행은 빼고 배열에 넣음
        continue
    line=line.split('\t')
    sentence.append(line[1])
    label.append(line[2].strip())#strip()은 주어진 문자열에서 양쪽 공백과 \n을 삭제시켜줌
    

In [56]:
import re #문장에서 불필요한 요소들 삭제

for i in range(len(sentence)):
    sentence[i] = sentence[i].lower()
    sentence[i] = re.sub("(<br\s*/><br\s*/>)|(/-)|(\/)",' ',sentence[i])
    sentence[i] = re.sub("[.;:!\'?,\"()\[\]]",' ',sentence[i])

In [57]:
#영화리뷰+직접 나눈 데이터를 train과 test로 적절하게 나눔
reviews_train_clean =sentence[:120000]
train_y=label[:120000]
reviews_test_clean = sentence[120000:]
test_y=label[120000:]

print(len(reviews_train_clean),len(train_y))
print(len(reviews_test_clean),len(test_y))

120000 120000
31056 31056


In [58]:
print(sentence[100:105])
print(label[100:105])

['사실상 신서유기가 1박2일 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ', '개콘개구맨들 좌퍼정권에 좀ㅂ들', '캐스팅이   연정훈은 노잼   문세윤 식상해    이건 아닌듯', '개콘 안본지 2년 넘었다 폐지해라', '정치풍자 요즘 왜 안하냐  할게 무궁무진 하잖아']
['1', '0', '0', '0', '0']


In [59]:
#데이터 전처리(문서를 숫자 벡터로 전환, 단어의 빈도수를 파악> 카운트백터로 나타냄)
from sklearn.feature_extraction.text import CountVectorizer

cv=CountVectorizer(binary=True)
cv.fit(reviews_train_clean)#train문장에서 모든 토큰의 어휘 사전을 학습
X=cv.transform(reviews_train_clean)#문장을 문장 용어 행렬로 변환
X_test = cv.transform(reviews_test_clean)#test문장을 문장 용어 행렬로 변환

In [60]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score #모델 성능 평가를 위한 라이브러리
from sklearn.model_selection import train_test_split #데이터를 나눈다

#train데이터를 또 다시 0.75:0.25비율의 train데이터와 test데이터로 나눔
X_train,X_val,y_train,y_val = train_test_split(X, train_y,test_size = 0.25, random_state=0)

for c in [0.01,0.05,0.25,0.5,1]:#정규화의 강도를 결정하는 하이퍼파라미터
    
    lr = LogisticRegression(C=c)
    lr.fit(X_train,y_train)#LogisticRegression 모델을 학습
    print("C=",c,", Accuracy :",accuracy_score(y_val,lr.predict(X_val)))#테스트본으로 성능 평가

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


C= 0.01 , Accuracy : 0.7425333333333334
C= 0.05 , Accuracy : 0.7755666666666666
C= 0.25 , Accuracy : 0.7974
C= 0.5 , Accuracy : 0.8028
C= 1 , Accuracy : 0.8059


In [61]:
final_model = LogisticRegression(C=1)
final_model.fit(X_train,y_train)#최적의 파라미터로 다시 학습

print("Final Accuracy: ",accuracy_score(test_y,final_model.predict(X_test)))#최종 테스트본으로 성능평가

Final Accuracy:  0.8077344152498712


# 댓글을 크롤링해온다

In [62]:
from bs4 import BeautifulSoup 
import os
import requests  
import re

os.chdir("C:/Project/text_project")#해당 디렉터리 지정
#댓글 가져오는 함수
def getcomment(url):
    List=[] 
    
    oid=url.split("oid=")[1].split("&")[0] 
    aid=url.split("aid=")[1] 
    page=1     
    header = { 
        "User-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36", 
        "referer":url, 
    } 
    i=1;
    while True : 
        c_url="https://apis.naver.com/commentBox/cbox/web_neo_list_jsonp.json?ticket=news&templateId=default_society&pool=cbox5&_callback=\
        jQuery1707138182064460843_1523512042464&lang=ko&country=&objectId=news"+oid+"%2C"+aid+"&categoryId=&pageSize=20&indexSize=10\
        &groupId=&listType=OBJECT&pageType=more&page="+str(page)+"&refresh=false&sort=FAVORITE"  
    # 파싱하는 단계입니다.
        r=requests.get(c_url,headers=header) 
        cont=BeautifulSoup(r.content,"html.parser")     
        total_comm=str(cont).split('comment":')[1].split(",")[0] #해당 페이지의 댓글개수
        print("진행상황:",i, end='\r')
        i=i+1;
        match=re.findall('"contents":([^\*]*),"userIdNo"', str(cont)) 
    # 댓글을 리스트에 중첩합니다.
        List.append(match) 
    # 한번에 댓글이 20개씩 보이기 때문에 한 페이지씩 몽땅 댓글을 긁어 옵니다.(마지막 페이지에서 빠져나옴)
        if int(total_comm) <= ((page) * 20): #만약 "if page>3:" 이면 4페이지까지의 댓글만 가져옴
            break
        else :
            page+=1
        
    return List #한 페이지가 한 배열로 저장돼 있다


# 여러 리스트들을 하나로 묶어 주는 함수입니다.
def flatten(l): 
    flatList = [] 
    for elem in l: 
        # 만약 elem이 list 타입이면 원소를 하나씩 빼와서 리스트에 저장 
        # elem이 리스트 타입이 아니면 그냥 리스트에 저장  
        if type(elem) == list: 
            for e in elem: 
                flatList.append(e) 
        else: 
            flatList.append(elem)
    return flatList



In [63]:
#크롤링해온 댓글을 긍정인지 부정인지 분별하고 긍정의 퍼센트를 구함
def Positive_Rate(comment):
    reviews_new_counts = cv.transform(comment) #cv=CountVectorizer(binary=True)

    pred = final_model.predict(reviews_new_counts) #훈련시킨 모델에 적용(긍정부정 예측)
    positive=0 #긍정적 반응 개수 새는 변수
    
        
    for pos in pred:#긍정적 반응 개수 샘
        if(pos=='1'):
            positive+=1
    positive_r=positive/len(pred)*100
          
    return len(pred), positive_r #총 댓글수, 총 댓글에서 긍정 반응의 비율

# 여기는 뉴스 본문요약, keyword 찾는 함수

In [64]:
from newspaper import Article
from konlpy.tag import Kkma #한국어 형태소 분석기
from konlpy.tag import Twitter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
import numpy as np

#[텍스트 크롤링, 문장 단위 분리, 명사 추출] 과정을 SentenceTokenizer 클래스로 만들었다.
class SentenceTokenizer(object):
    def __init__(self):
        self.kkma = Kkma()
        self.twitter = Twitter()
        self.stopwords = ['중인' ,'만큼', '마찬가지', '꼬집었', "연합뉴스", "데일리", "동아일보", "중앙일보", "조선일보", "기자"
,"아", "휴", "아이구", "아이쿠", "아이고", "어", "나", "우리", "저희", "따라", "의해", "을", "를", "에", "의", "가", "서울신문", "연합뉴스"]
    def url2sentences(self, url): #url 주소를 받아 기사내용(article.text)을 추출하여 문장단위로 나누어 준 후 senteces를 return 해 준다.
        article = Article(url, language='ko')
        article.download()
        article.parse()
        sentences = self.kkma.sentences(article.text)#기사내용을 문장단위로 나눔
        
        for idx in range(0, len(sentences)): #길이가 작은 문장은 앞 문장과 합친다.
            if len(sentences[idx]) <= 10:
                sentences[idx-1] += (' ' + sentences[idx])
                sentences[idx] = ''
        return sentences
    def text2sentences(self, text): #text(str)를 입력받아 문장단위로 나누어 준 후 senteces를 return 해 준다.
        sentences = self.kkma.sentences(text) #Kkma.sentences()를 이용하여 문장단위로 나누어 줌
        for idx in range(0, len(sentences)):
            if len(sentences[idx]) <= 10:
                sentences[idx-1] += (' ' + sentences[idx])
                sentences[idx] = ''
        return sentences
    def get_nouns(self, sentences): #sentences를 받아 명사를 추출한 뒤 nouns를 return해 준다.
        nouns = []
        for sentence in sentences:
            if sentence is not '':
                nouns.append(' '.join([noun for noun in self.twitter.nouns(str(sentence)) #Twitter.nouns()를 이용하여 명사를 추출
                    if noun not in self.stopwords and len(noun) > 1]))
        return nouns


In [65]:
class GraphMatrix(object):
    def __init__(self):
        self.tfidf = TfidfVectorizer()
        self.cnt_vec = CountVectorizer()
        self.graph_sentence = []
    def build_sent_graph(self, sentence):
        tfidf_mat = self.tfidf.fit_transform(sentence).toarray()
        self.graph_sentence = np.dot(tfidf_mat, tfidf_mat.T)
        
        return self.graph_sentence
    
    def build_words_graph(self, sentence):
        cnt_vec_mat = normalize(self.cnt_vec.fit_transform(sentence).toarray().astype(float), axis=0)
        
        vocab = self.cnt_vec.vocabulary_
        return np.dot(cnt_vec_mat.T, cnt_vec_mat), {vocab[word] : word for word in vocab}




In [66]:
#{idx : rank 값} 형태의 dictionary를 return 한다.
class Rank(object):
    def get_ranks(self, graph, d=0.85): # d = damping factor
        A = graph
        matrix_size = A.shape[0]
        for id in range(matrix_size):
            A[id, id] = 0 # diagonal 부분을 0으로
            link_sum = np.sum(A[:,id]) # A[:, id] = A[:][id]
            if link_sum != 0:
                A[:, id] /= link_sum
            A[:, id] *= -d
            A[id, id] = 1
        B = (1-d) * np.ones((matrix_size, 1))
        ranks = np.linalg.solve(A, B) # 연립방정식 Ax = b
        
        return {idx: r[0] for idx, r in enumerate(ranks)}


In [67]:
#TextRank 알고리즘을 통해 Ranking값이 높은 순으로 정렬한 뒤 요약할 문장(or 단어)의 개수 만큼 출력 해준다.
class TextRank(object):
    def __init__(self, text): #text매개 변수를 sentence로 나눔
        self.sent_tokenize = SentenceTokenizer()
        if text[:5] in ('http:', 'https'): #매개변수 text가 url일때
            self.sentences = self.sent_tokenize.url2sentences(text)
        else: #매개변수 text가 text일때
            self.sentences = self.sent_tokenize.text2sentences(text)
        self.nouns = self.sent_tokenize.get_nouns(self.sentences)
        
        self.graph_matrix = GraphMatrix()
        self.sent_graph = self.graph_matrix.build_sent_graph(self.nouns)
        self.words_graph, self.idx2word = self.graph_matrix.build_words_graph(self.nouns)
        
        self.rank = Rank()
        self.sent_rank_idx = self.rank.get_ranks(self.sent_graph)
        self.sorted_sent_rank_idx = sorted(self.sent_rank_idx, key=lambda k: self.sent_rank_idx[k], reverse=True)
        
        self.word_rank_idx = self.rank.get_ranks(self.words_graph)
        self.sorted_word_rank_idx = sorted(self.word_rank_idx, key=lambda k: self.word_rank_idx[k], reverse=True)
    def summarize(self, sent_num=3):#뉴스본문내용 요약하는 함수

        summary = []
        index=[]
        for idx in self.sorted_sent_rank_idx[:sent_num]:
            index.append(idx)
        index.sort()
        for idx in index:
            summary.append(self.sentences[idx])
        return summary
    def keywords(self, word_num=1):#키워드 추출하는 함수(word_num은 추출할 키워드 개수) 
        rank = Rank()
        rank_idx = rank.get_ranks(self.words_graph)
        sorted_rank_idx = sorted(rank_idx, key=lambda k: rank_idx[k], reverse=True)#내림차순 정렬
        keywords = []
        index=[]
        for idx in sorted_rank_idx[:word_num]:
            index.append(idx)
        
        #index.sort()
        for idx in index:
            keywords.append(self.idx2word[idx])
            
        return keywords

        



# 랭킹뉴스의 종목(정치), 제목(헤드라인), 신문사, url을 크롤링 해와서 엑셀파일에 저장하고 문서요약, keyword도 엑셀에 저장

In [68]:
#날짜별 많이본 랭킹 뉴스 종목(정치), 제목(헤드라인), 신문사, url 등 불러오기 
from urllib.request import urlopen
from bs4 import BeautifulSoup
import csv

news_day=eval(input('날짜를 입력하세요(ex:20191119) : '))
section_list=[100,101,102,103,104,105] #정치, 경제, 사회 등을 가져오기 위해 url에 넣을 변수들

titleList=[]
titleList.append(['분야','헤드라인','신문사','뉴스url','본문요약','keyword','총 댓글수', '긍정적 댓글비율(%)'])

while(True): #데이터를 가져올 분야 설정구간(정치, 경제, 사회.. 순으로 데이터 가져온다)
    for section in section_list:
        if section==100:
            section_word="정치"
            section_list.remove(100)
            break
        elif section==101:
            section_word="경제"
            section_list.remove(101)
            break
        elif section==102:
            section_word="사회"
            section_list.remove(102)
            break
        elif section==103:
            section_word="생활/문화"
            section_list.remove(103)
            break
        elif section==104:
            section_word="세계"
            section_list.remove(104)
            break
        elif section==105:
            section_word="IT/과학"
            section_list.remove(105)
            break


    url = f'https://news.naver.com/main/ranking/popularDay.nhn?rankingType=popular_day&sectionId={section}&date={news_day}'
    #날짜별 랭킹뉴스 url

    html=urlopen(url).read()
    soup=BeautifulSoup(html, 'html.parser') #html파싱
    
    #엑셀에 넣을 데이터들
    for li in soup.select('div.content > div.ranking > ol > li > div.ranking_text'):#분야에 따른 헤드라인 리스트
        temp=[]
    
        news_url="https://news.naver.com"+li.find('a').attrs['href']#랭킹뉴스에서 하나의 뉴스로 들어가기 위해 url전처리
        news_url.replace("amp;","") # amp;는 지워줘야함    
    
        temp.append(section_word)#분야
        temp.append(li.find('a').attrs['title'])#헤드라인 제목
        temp.append(li.select_one('div[class="ranking_office"]').text)#신문사(조선일보 등등)      
    
        temp.append(news_url)#뉴스url    
        textrank = TextRank(news_url)#문서 요약,keyword 알아내는 함수를 호출
        temp.append(textrank.summarize(3))#문서 요약본{idx : rank 값} 형태의 dictionary를 return 한다.


        temp.append(textrank.keywords())#keywords
    
        comment=flatten(getcomment(news_url))#뉴스 댓글들 호출 함수(배열로 저장)
        comment_num, positive_r=Positive_Rate(comment)#댓글들의 긍정부정 검사
        temp.append(comment_num)#댓글의 개수
        temp.append(positive_r)#긍적적 댓글의 비율    
        
        titleList.append(temp)
    
    if(section_word=="IT/과학"):
        break
        
    
#엑셀 파일로 저장   
search=input('저장할 파일 이름을 입력하세요 : ')
f =open(f'{search}.csv', 'w', encoding='utf-8', newline='')
csvWriter = csv.writer(f)
for i in titleList:
    csvWriter.writerow(i)

f.close()
print('완료되었습니다.')



날짜를 입력하세요(ex:20191119) :  20191202


C:\ProgramData\Anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


C:\ProgramData\Anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


C:\ProgramData\Anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


C:\ProgramData\Anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


...

C:\ProgramData\Anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


C:\ProgramData\Anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


C:\ProgramData\Anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


저장할 파일 이름을 입력하세요 :  20191202_뉴스


완료되었습니다.
